Bloc 1: Instalació de llibreries  
Bloc 2: Importació de llibreries  
Bloc 3: Per treballar amb fitxers des del Drive  
Bloc 4: Definició de funcions  
Bloc 5: Crida de funcions  
  
Com fer servir:  
- Executar bloc 1 i després bloc 2  
- Executar bloc 4  
SI ES VOL TREBALLAR AL DRIVE:  
+ Executar el bloc 3 i introduïr el codi de la web
+ Canviar les variables "csv_datab_name" i "filename" a la carpeta on hi ha la base de dades  
SI NO:  
+ Canviar les variables "csv_datab_name" i "filename" a la carpeta on hi ha la base de dades  
FINALMENT:
- Executar el bloc 5  
- Es pot treballar amb les dades fasta al bloc 6

In [0]:
#Bloc 1

!pip3 install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 74.6 kB in 2s (37.4 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree    

In [0]:
#Bloc 2

import csv
import timeit
import math
import requests
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver

In [0]:
#Bloc 3

from google.colab import drive
drive.mount('/content/drive')

In [0]:
#Bloc 4

def get_global_data(route):
    #Format global_data: Accession, Length, Location, Date
    global_data = []
    with open(route,newline="") as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for i, row in enumerate(reader):
            if i > 3:
                line = ','.join(row).split(",")
                if line[2].endswith(":"):
                    line[2] = line[2][:-1]
                    line[3] = line[4]
                    line = line[:-1]
                if line[2].startswith("\""):
                    line[2] = line[2][1:]
                if line[2] != '':
                    global_data.append(line)

    return global_data

def separate_in_countries(data,mode):
    if mode == 1:
        countries = []
        for row in data:
            if not row[2] in countries:
                countries.append(row[2])
        separated_data = []
        for country in countries:
            country_line = []
            for row in data:
                if row[2] == country:
                    country_line.append(row)
            separated_data.append(country_line)
        return separated_data
    else:
        separated_data = {}
        for row in data:
            if row[2] in separated_data.keys():
                separated_data[row[2]].append(row)
            else:
                separated_data[row[2]] = [row]
        return [separated_data[key] for key in separated_data.keys()]

def country_medians(data):
    medians = []
    for country_data in data:
        country_name = country_data[0][2]
        medians.append((country_name,country_data[math.floor(len(country_data) / 2)][1],country_data[math.floor(len(country_data) / 2)][0]))
    return medians

def get_files(country_data):
    url = "https://www.ncbi.nlm.nih.gov/nuccore/" + str(country_data[2]) + ".1?report=fasta"
    filename = str(country_data[0]) + ".fasta"

    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get(url)
    sleep(0.2)
    soup = BeautifulSoup(driver.page_source, 'lxml')

    f = open(filename,"w")
    f.write("".join(soup.find(id="viewercontent1").text.split("\n")[1:]))
    f.close()

In [63]:
#Bloc 5

#Ruta Joel A, l'haureu de canviar si ho voleu provar
csv_datab_route = "/content/drive/My Drive/Uni/Practica1-Algoritmica/all_sequences.csv"
file_route = "/content/drive/My Drive/Uni/Practica1-Algoritmica/time.txt"
f = open(file_route,"w")

#O(N) -> N = Numero total de mostres
my_data = get_global_data(csv_datab_route)
f.write("Get global data:\n")
f.write(str(timeit.timeit("get_global_data(csv_datab_route)","from __main__ import csv_datab_route, get_global_data",number=1)) + "\n")

#O(M*N) -> M = Numero total de països
my_countries = separate_in_countries(my_data,2)
f.write("Separate in countries 1:\n")
f.write(str(timeit.timeit("separate_in_countries(my_data,1)","from __main__ import separate_in_countries,my_data",number=1)) + "\n")
f.write("Separate in countries 2:\n")
f.write(str(timeit.timeit("separate_in_countries(my_data,2)","from __main__ import separate_in_countries,my_data",number=1)) + "\n")

#O(M)
medians = country_medians(my_countries)
f.write("Medians:\n")
f.write(str(timeit.timeit("country_medians(my_countries)","from __main__ import country_medians,my_countries",number=1)) + "\n")

#O(M)
total_timeit = 0.0
for data in medians:
    get_files(data)
    total_timeit += timeit.timeit("get_files(data)","from __main__ import get_files,data",number=1)

f.write("Total file scraping: ")
f.write(str(total_timeit))
f.write("Average file scraping: ")
f.write(str(total_timeit/len(medians)))

print("DONE")

f.close()

DONE


In [0]:
#Bloc 6

def sequence_alignment():
  sequence_1 = "ACCGAAC"
  sequence_2 = "ACCTAAC"
  match = -1
  non_match = 4
  gap = 5
  result = 0
  matriu = []

  #Creació matriu
  for _ in range(len(sequence_1) + 2):
    fila = []
    for _ in range(len(sequence_2) + 2):
      fila.append(0)
    matriu.append(fila)

  #Inicialització dels eixos
  matriu[0][1] = "-"
  matriu[1][0] = "-"

  for i in range(len(sequence_1)):
    matriu[i + 2][0] = sequence_1[i]
  for i in range(len(sequence_2)):
    matriu[0][i + 2] = sequence_2[i]

  #Omplim la matriu
  for i = 1 in range(len(sequence_1)):
    for j = 0 in range(len(sequence_2)):
      if (sequence_1[i] == sequence_2[j]):